# Welding Anomaly Detection with ResNet-50 Binary Classifier

This notebook demonstrates a complete pipeline for welding anomaly detection using:
1. **YOLO** for weld region detection
2. **ResNet-50** fine-tuned as a binary classifier for good/bad weld classification

## Data Split Strategy
- **Training Set**: Used for model training and parameter updates
- **Validation Set**: Used for model selection (early stopping) AND final evaluation
- **No Test Set**: Validation set serves dual purpose for simplicity

## Pipeline Overview
- Extract weld ROIs from images using XML annotations
- Create a balanced dataset with proper train/val splits  
- Fine-tune ResNet-50 with careful data preprocessing and augmentation
- Evaluate using ROC-AUC, PR-AUC, and F1 metrics on validation set

⚠️ **Note**: In production, consider using a separate test set for unbiased final evaluation.

In [1]:
!pip install -r requirements.txt

In [1]:
import torch
from pathlib import Path
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("Using CPU")

CUDA available: True
GPU: NVIDIA L4
CUDA version: 12.1


In [ ]:
!python cnn_train/yolo/dataset_formatter.py

In [ ]:
!python cnn_train/yolo/yolo_train.py

In [5]:
# Check if we have the required data structure
base_dir = Path('/teamspace/studios/this_studio')
train_dir = base_dir / 'downloaded_photos' / 'train'
val_dir = base_dir / 'downloaded_photos' / 'val'

if train_dir.exists():
    train_images = list(train_dir.glob('*.jpeg')) + list(train_dir.glob('*.jpg'))
    train_xmls = list(train_dir.glob('*.xml'))
    print(f"Training images: {len(train_images)}")
    print(f"Training XMLs: {len(train_xmls)}")

if val_dir.exists():
    val_images = list(val_dir.glob('*.jpeg')) + list(val_dir.glob('*.jpg'))
    val_xmls = list(val_dir.glob('*.xml'))
    print(f"Validation images: {len(val_images)}")
    print(f"Validation XMLs: {len(val_xmls)}")

# Run the weld extraction script
!python cnn_train/cnn/extract_welds.py

Training images: 2055
Training XMLs: 2055
Validation images: 870
Validation XMLs: 870
=== Welding Region Extraction ===
Processing training set: /teamspace/studios/this_studio/downloaded_photos/train
Extracting welding patches: 100%|███████████| 2055/2055 [00:31<00:00, 64.24it/s]

Extraction completed!
Total patches extracted: 11300
Good welds: 8481
Bad welds: 2819
Patches saved to: cnn_train/cnn/extracted_welds
Processing validation set: /teamspace/studios/this_studio/downloaded_photos/val
Extracting welding patches: 100%|█████████████| 870/870 [00:15<00:00, 54.75it/s]

Extraction completed!
Total patches extracted: 4800
Good welds: 3749
Bad welds: 1051
Patches saved to: cnn_train/cnn/extracted_welds


In [2]:
# Step 3: Train ResNet-50 Binary Classifier
!python cnn_train/cnn/welding_cnn.py --mode "train"

Using device: cuda
=== Training ResNet-50 Welding Classifier ===
Device: CUDA
Using train/validation split (no separate test set)

Training Configuration:
  epochs: 50
  batch_size: 32
  patience: 8
  save_dir: cnn_train/cnn/runs
  val_interval_steps: 50

=== Starting Training ===
Balancing dataset: using 2819 samples per class
Found 2819 images in cnn_train/cnn/extracted_welds/train/good
Found 2819 images in cnn_train/cnn/extracted_welds/train/bad
Found 3749 images in cnn_train/cnn/extracted_welds/val/good
Found 1051 images in cnn_train/cnn/extracted_welds/val/bad
Total parameters: 25,557,032
Trainable parameters before freezing: 25,557,032
Trainable parameters after freezing: 22,065,153
Model initialized with 22,065,153 trainable parameters

=== Starting Training ===
Epochs: 50, Batch Size: 32, Patience: 8
Validation Interval: Every 50 steps

Epoch 1/50
Training:  28%|▎| 49/177 [00:18<00:34,  3.74it/s, batch_loss=0.212, avg_loss=0.4
--- Validation at step 50 ---

Validating: 100%|███

In [ ]:
!python cnn_train/cnn/welding_cnn.py --mode "val"